In [111]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from torchvision.models import resnet18, ResNet18_Weights
import torch
import PIL

In [112]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
import numpy as np
from PIL import Image



# DATASET_PATH = '/kaggle/input/dog-emotion/Dog Emotion/'
DATASET_PATH = '/content/drive/MyDrive/Dog_Emotion_1/'

data = {'img': [], 'emotion': []}

for emotion in ['angry', 'happy', 'relaxed', 'sad']:
    for dirname, _, filenames in os.walk(DATASET_PATH + emotion + '/'):
        for filename in filenames:
            data['img'].append(os.path.join(dirname, filename))
            data['emotion'].append(emotion)

df = pd.DataFrame.from_dict(data)

In [114]:
df

,img,emotion
0,/content/drive/MyDrive/Dog_Emotion_1/angry/0Av...,angry
1,/content/drive/MyDrive/Dog_Emotion_1/angry/0C5...,angry
2,/content/drive/MyDrive/Dog_Emotion_1/angry/09d...,angry
3,/content/drive/MyDrive/Dog_Emotion_1/angry/0TS...,angry
4,/content/drive/MyDrive/Dog_Emotion_1/angry/0aN...,angry
...,...,...
4005,/content/drive/MyDrive/Dog_Emotion_1/sad/zvgEE...,sad
4006,/content/drive/MyDrive/Dog_Emotion_1/sad/ztwzd...,sad
4007,/content/drive/MyDrive/Dog_Emotion_1/sad/zmAi6...,sad
4008,/content/drive/MyDrive/Dog_Emotion_1/sad/zq3lf...,sad


In [115]:
emotion_labels_map = {}

unique_emotions = df['emotion'].unique()
for i in range(len(unique_emotions)):
    emotion_labels_map[unique_emotions[i]] = i

In [116]:
df['emotion_label'] = df['emotion'].apply(lambda x: emotion_labels_map[x])
df

,img,emotion,emotion_label
0,/content/drive/MyDrive/Dog_Emotion_1/angry/0Av...,angry,0
1,/content/drive/MyDrive/Dog_Emotion_1/angry/0C5...,angry,0
2,/content/drive/MyDrive/Dog_Emotion_1/angry/09d...,angry,0
3,/content/drive/MyDrive/Dog_Emotion_1/angry/0TS...,angry,0
4,/content/drive/MyDrive/Dog_Emotion_1/angry/0aN...,angry,0
...,...,...,...
4005,/content/drive/MyDrive/Dog_Emotion_1/sad/zvgEE...,sad,3
4006,/content/drive/MyDrive/Dog_Emotion_1/sad/ztwzd...,sad,3
4007,/content/drive/MyDrive/Dog_Emotion_1/sad/zmAi6...,sad,3
4008,/content/drive/MyDrive/Dog_Emotion_1/sad/zq3lf...,sad,3


In [117]:
from torch.utils.data import DataLoader, Dataset

class Dataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = PIL.Image.open(self.data.loc[idx, "img"]).convert('RGB')
        image = self.transform(image)
        label = torch.tensor(self.data.loc[idx, "emotion_label"])
        return image, label

In [118]:
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["img"], df["emotion_label"], random_state=42)

X_train.index = np.arange(len(X_train))
y_train.index = np.arange(len(y_train))
X_test.index = np.arange(len(X_test))
y_test.index = np.arange(len(y_test))

In [234]:
transform_test = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [264]:
transform_train = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(degrees=7),
    v2.ColorJitter(brightness=0.2, contrast=0.2),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [265]:
train_ds = Dataset(pd.concat([X_train, y_train], axis=1), transform_train)
test_ds = Dataset(pd.concat([X_test, y_test], axis=1), transform_test)

train_loader = DataLoader(train_ds, batch_size=32, num_workers=4)
test_loader = DataLoader(test_ds, batch_size=32, num_workers=4)

In [266]:
class Extraction:
    def __init__(self, network):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.network = network.eval().to(self.device)

    def extract(self, loader):
        data_tmp = []
        label_tmp = []

        with torch.no_grad():
            for x, y in loader:
                x = x.to(self.device)

                outputs = self.network(x)
                data_tmp.append(outputs.view(-1, 2048).cpu().numpy())

                label_tmp.append(y.cpu().numpy())

        return np.vstack(data_tmp), np.hstack(label_tmp)

In [267]:
from torchvision.models import resnet50, ResNet50_Weights

model = resnet50(weights=ResNet50_Weights.DEFAULT)

In [276]:
from torchvision.models import resnet18, efficientnet_b0, ResNet18_Weights, EfficientNet_B0_Weights
import torch

resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
efficientnet = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)

resnet_feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])
efficientnet_feature_extractor = torch.nn.Sequential(*list(efficientnet.children())[:-1])

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 117MB/s] 


In [277]:
class MultiModelExtraction:
    def __init__(self, networks):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.networks = [network.eval().to(self.device) for network in networks]

    def extract(self, loader):
        data_tmp = []
        label_tmp = []

        with torch.no_grad():
            for x, y in loader:
                x = x.to(self.device)
                features = []
                for network in self.networks:
                    output = network(x)
                    features.append(output.view(output.size(0), -1).cpu().numpy())
                combined_features = np.hstack(features)
                data_tmp.append(combined_features)
                label_tmp.append(y.cpu().numpy())

        return np.vstack(data_tmp), np.hstack(label_tmp)

In [278]:
multi_ext = MultiModelExtraction([resnet_feature_extractor, efficientnet_feature_extractor])
train_feature_multi, train_label_multi = multi_ext.extract(train_loader)
test_feature_multi, test_label_multi = multi_ext.extract(test_loader)

In [369]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

param = {
    "kernel": ['rbf'],
    'C': [5, 7, 8, 10]
}

svc = svm.SVC()
svm_grid = GridSearchCV(svc, param_grid=param, verbose=2, n_jobs=-1)

svm_grid.fit(train_feature_multi, train_label_multi)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [5, 7, 8, 10], 'kernel': ['rbf']}, verbose=2)

In [370]:
best_params = svm_grid.best_params_

In [371]:
print("Лучшие параметры:", best_params)

Лучшие параметры: {'C': 5, 'kernel': 'rbf'}


In [372]:
y_pred_svm_grid = svm_grid.predict(test_feature_multi)

In [373]:
from sklearn.metrics import accuracy_score

accuracy_score(test_label_multi, y_pred_svm_grid)

0.7896311066799602